<a href="https://colab.research.google.com/github/timsetsfire/wandb-examples/blob/main/colab/XGBoost_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this lab is to instrument W&B a top of existing ML Workflows which might be leveraging

* XGBoost
* Pandas
* Shap
* STDOUT (for metric tracking)

We will augment this workflow by leveraging

* Wandb Experiments
* Wandb logging
* Wandb Artifacts for dataset and model logging / versioning
* Tables to surface prediction examples on Test datasets
track lineage of all artifacts and experiments completed

Lastly, we'll do a simple demo of sweeps and interact with the runs via W&B API to

query runs and run summaries
artifacts

In [34]:
# %%capture
!pip install wandb xgboost==1.3.3 shap xlrd==1.2.0 category_encoders -qqq

In [15]:
# General imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
import json
from pathlib import Path
import wandb

from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import xgboost as xgb
from wandb.xgboost import WandbCallback
import wandb

## Logging in

You can login to W&B Anonymously.  Which means, you do NOT need to create an account, and you will be able to run this notebook entirely.  

Be advised that artifacts logged anonymously cannot be claimed and expire after 7 days.

In [16]:
#@title Enter host address
#@markdown Enter the host url which corresponds to your WB instance.
host = "https://api.wandb.ai" #@param {type: "string"}
## when using wandb anywhere other than wandb.ai, you must
## provide a proper host, so the client knows where to communcate
## details of the experiment
# wandb.login(key = key, host = host)
wandb.login(host = host)

True

## The UCI Credit-card Default Dataset

The UCI dataset contains data on 30,000 clients and their credit card transactions at a bank in Taiwan. In addition to static client features, the dataset contains the history of credit card bill payments between April and September 2005, as well as the balance limit of the client's credit card. The target is whether the client will default on a card payment in the following month, October 2005. A model trained on this data could be used, in part, to determine whether a client is eligible for another loan or a credit increase.

## Get Data (and log it)

When we get data and log it, there are obviously tons of way to complete this.  Depending on how you log data, and whether or not you log your retrieval mechanism is a matter of preference and internal guidelines you need to follow.  

Before we get started it is important to set the name space for your project.  This is going to be accomplished by passing a `project_name` as well as an `entity` to your wandb experiment

`entity` corresponds to the team to which the project will be associated with.  The `entity` could be a team name, or your user name.  

In [17]:
project_name = "xgboost-example" #@param {type: "string"}
entity = "tim-w" #@param {type: "string"}

## Logging data

W&B is very unopinionated with regard to how you track your experiments.  We could log data in any number of ways.  
* Log one artifact which represents all the data - training, validation, and test data to one artifact
* Log several artifacts - one for each of the training, validation, and test data loaders.  

It is a matter of what best suites your needs and workflows and expectations.  

### Anatomy of an artifact

The `Artifact` class will correspond to an entry in the W&B Artifact registry.  The artifact has
* a name
* a type
* metadata
* description
* files, directory of files, or references

Example usage
```
run = wandb.init(project = "my-project")
artifact = wandb.Artifact(name = "my_artifact", type = "data")
artifact.add_file("/path/to/my/file.txt")
run.log_artifact(artifact)
run.finish()
```

In [18]:
# get and log data
with wandb.init(project = project_name, job_type = "data-acquisition") as run:
  data_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
  dataset = pd.read_excel(io=data_url, header=1).drop(columns=['ID']).rename(columns={'PAY_0':'PAY_1'})
  dataset.head()
  data_artifact = wandb.Artifact(name = "raw-dataset",  type = "dataset", incremental = True)
  data_artifact.add_reference(data_url)   ## s3 bucket, nfs drive
  run.log_artifact(data_artifact)

wandb: WARNING Using experimental arg `incremental`


Dataset columns:

* `LIMIT_BAL`: credit card limit, will be replaced by a synthetic feature
* `SEX, EDUCATION, MARRIAGE, AGE`: client demographic features
* `BILL_AMT[1-6]`: amount on bill statement for April-September
* `PAY_AMT[1-6]`: payment amount for April-September
* `default payment next month`: target, whether the client defaulted the following month

In [19]:
from pathlib import Path
data_path = Path("./data")
data_path.mkdir(exist_ok = True)
target_name = "default payment next month"

In [21]:
# Train-test split
with wandb.init(project = project_name, job_type = "data-split") as run:
  test_size = 0.3
  random_state = 12345
  dataset_artifact = run.use_artifact(f'{entity}/{project_name}/raw-dataset:latest', type='dataset')
  #
  dataset_artifact.download()
  #
  df = pd.read_excel(dataset_artifact.file())
  #

  df_train, df_test = train_test_split(
      dataset,
      test_size = 0.1,
      random_state=12345,
      stratify=dataset[target_name])
  df_train, df_val = train_test_split(
      df_train,
      test_size = 0.2,
      random_state=12345,
      stratify=df_train[target_name])
  df_train.to_csv("data/train.csv", index = False)
  df_test.to_csv("data/test.csv", index = False)
  df_val.to_csv("data/val.csv", index = False)
  training_data_artifact = wandb.Artifact(name = "train-val-test-datasets",
                                          type = "dataset",
                                          description = "train/validation/test data",
                                          metadata = {"stratified": target_name, "valsize": 0.2, "testsize": 0.1, "trainsize": 0.7, "random_state":12345})
  training_data_artifact.add_dir("./data")
  run.log_artifact(training_data_artifact)

wandb:   1 of 1 files downloaded.  
wandb: Adding directory to artifact (./data)... Done. 0.0s


## Create Pipeline

In [22]:
categorical_features = ["EDUCATION", "MARRIAGE", "PAY_1", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"]
numeric_features = ['LIMIT_BAL', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3',
      'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2',
      'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
features = []
features.extend(numeric_features)
features.extend(categorical_features)
for col in categorical_features:
  dataset[col] = dataset[col].astype('category')


X_train = df_train.drop([target_name], axis=1)
Y_train = df_train[target_name]
X_test = df_test.drop([target_name], axis=1)
Y_test = df_test[target_name]
X_val = df_val.drop([target_name], axis=1)
Y_val = df_val[target_name]
base_rate = Y_train.mean()

In [23]:
## create pipeline

def create_preprocessing_pipeline(num_features, cat_features):
  cat_preprocessing = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value=-9999, add_indicator=False)),
        ("category_encoder", OrdinalEncoder()),
    ]
  )


  pipeline = Pipeline([
      ("selector", ColumnTransformer(
          [("selector", "passthrough", numeric_features),
          ("categorical-encoding", cat_preprocessing, categorical_features)
          ], remainder="drop")),
  ])

  pipeline = Pipeline(steps=[("preprocessing", pipeline)])

  return pipeline


In [24]:
from wandb.xgboost import WandbCallback



In [25]:
# Initialize the XGBoostClassifier
import sys

config = {
        'objective': 'binary:logistic'
        , 'base_score': base_rate
        , 'gamma': 1               ## def: 0
        , 'learning_rate': 0.1     ## def: 0.1
        , 'max_depth': 3
        , 'min_child_weight': 100  ## def: 1
        , 'n_estimators': 25
        , 'nthread': 24
        , 'random_state': 42
        , 'reg_alpha': 0
        , 'reg_lambda': 0          ## def: 1
        , 'eval_metric': ['auc', 'logloss']
        , 'tree_method': 'hist'  # use `gpu_hist` to train on GPU
    }

run = wandb.init(project=project_name, job_type="training", config = config)

# artifact = run.use_artifact('{entity}/{project_name}/train-val-test-datasets:v0', type='dataset')
artifact = run.use_artifact('tim-w/xgboost-example/train-val-test-datasets:v0', type='dataset')
## download and load if necessary

early_stopping_rounds = 40

pipeline = create_preprocessing_pipeline(numeric_features, categorical_features)
pipeline.fit(X_train)

X_train = pipeline.transform(X_train)
X_val = pipeline.transform(X_val)
X_test = pipeline.transform(X_test)

X_train = pd.DataFrame(X_train, columns = features)
X_val = pd.DataFrame(X_val, columns = features)
X_test = pd.DataFrame(X_test, columns = features)

xgbmodel = xgb.XGBClassifier(**wandb.config, use_label_encoder=False)
xgbmodel.fit(X_train, Y_train, eval_set = [(X_val, Y_val)],
             callbacks = [WandbCallback(log_model = True)]
             )

run.finish()

[0]	validation_0-auc:0.74603	validation_0-logloss:0.50882
[1]	validation_0-auc:0.75616	validation_0-logloss:0.49454
[2]	validation_0-auc:0.76581	validation_0-logloss:0.48310


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[3]	validation_0-auc:0.76847	validation_0-logloss:0.47404
[4]	validation_0-auc:0.77308	validation_0-logloss:0.46673
[5]	validation_0-auc:0.77510	validation_0-logloss:0.46052
[6]	validation_0-auc:0.77469	validation_0-logloss:0.45557
[7]	validation_0-auc:0.77533	validation_0-logloss:0.45137
[8]	validation_0-auc:0.77599	validation_0-logloss:0.44772
[9]	validation_0-auc:0.77758	validation_0-logloss:0.44432
[10]	validation_0-auc:0.77809	validation_0-logloss:0.44169
[11]	validation_0-auc:0.77874	validation_0-logloss:0.43956
[12]	validation_0-auc:0.77915	validation_0-logloss:0.43754
[13]	validation_0-auc:0.78006	validation_0-logloss:0.43594
[14]	validation_0-auc:0.77978	validation_0-logloss:0.43465
[15]	validation_0-auc:0.77970	validation_0-logloss:0.43334
[16]	validation_0-auc:0.78040	validation_0-logloss:0.43214
[17]	validation_0-auc:0.78065	validation_0-logloss:0.43115
[18]	validation_0-auc:0.78065	validation_0-logloss:0.43036
[19]	validation_0-auc:0.78075	validation_0-logloss:0.42967
[20]

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
validation_0-auc,▁▃▅▅▆▇▇▇▇▇▇▇▇██▇█████████
validation_0-logloss,█▇▆▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch,24


In [28]:
import shap

with wandb.init(project=project_name, job_type="training", config = config) as run:

  # artifact = run.use_artifact('{entity}/{project_name}/train-val-test-datasets:v0', type='dataset')
  artifact = run.use_artifact('tim-w/xgboost-example/train-val-test-datasets:v0', type='dataset')
  ## download and load if necessary

  early_stopping_rounds = 40

  pipeline = create_preprocessing_pipeline(numeric_features, categorical_features)
  pipeline.fit(X_train)

  X_train = pipeline.transform(X_train)
  X_val = pipeline.transform(X_val)
  X_test = pipeline.transform(X_test)

  X_train = pd.DataFrame(X_train, columns = features)
  X_val = pd.DataFrame(X_val, columns = features)
  X_test = pd.DataFrame(X_test, columns = features)

  xgbmodel = xgb.XGBClassifier(**wandb.config, use_label_encoder=False)
  xgbmodel.fit(X_train, Y_train, eval_set = [(X_val, Y_val)],
              callbacks = [WandbCallback(log_model = True)])

  explainer = shap.TreeExplainer(xgbmodel)
  shap_values = explainer.shap_values(X_test)

  Y_test_probs = xgbmodel.predict_proba(X_test)
  Y_test_preds = Y_test_probs[:,1]

  y_preds = pd.DataFrame(Y_test_probs, columns = ["p0", "p1"])
  sorted_idx = y_preds.sort_values("p1").index

  shap_html_list = [np.nan]*y_preds.shape[0]

  ## log a few shap plots for top and bottom predictions

  for i in sorted_idx[0:1]:
    force_plot = shap.force_plot(explainer.expected_value, shap_values[i,:], X_test.iloc[i,:], show=False, matplotlib=False)
    shap_html = f"<head>{shap.getjs()}</head><body>{force_plot.html()}</body>"
    shap_html_list[i] = wandb.Html(shap_html)
    run.log({f"prediction explanation at idx: {i}, prob: {y_preds.iloc[i][0]}": wandb.Html(shap_html)})

  for i in sorted_idx[-1:]:
    force_plot = shap.force_plot(explainer.expected_value, shap_values[i,:], X_test.iloc[i,:], show=False, matplotlib=False)
    shap_html = f"<head>{shap.getjs()}</head><body>{force_plot.html()}</body>"
    shap_html_list[i] = wandb.Html(shap_html)
    run.log({f"prediction explanation at idx: {i}, prob: {y_preds.iloc[i][0]}": wandb.Html(shap_html)})

  ## log stacked shap plot for first 1000 records in test dataset
  summary_force_plot = shap.force_plot(explainer.expected_value, shap_values[:1000,:], X_test.iloc[:1000,:])
  shap_html = f"<head>{shap.getjs()}</head><body>{summary_force_plot.html()}</body>"
  run.log({f"aggregate ": wandb.Html(shap_html)})

  ## feature importance and partial dependence plots
  shap_values_df = pd.DataFrame(shap_values, columns = [f"{c}_shap" for c in X_test.columns])

  shap_fi = np.abs(shap_values_df).mean(axis=0).reset_index()
  shap_fi.columns = ["Feature", "Feature Importance"]
  shap_fi["Feature"] = shap_fi["Feature"].apply(lambda x: x[0:-5])
  shap_wandb_table = wandb.Table(dataframe = shap_values_df.join(df_test.reset_index()))
  shap_fi_plot = wandb.plot.bar(table =  wandb.Table(dataframe = shap_fi), label = "Feature", value = "Feature Importance", title = "Shap Feature Importance")
  top_5 = shap_fi.sort_values("Feature Importance", ascending = False).iloc[0:5]["Feature"].tolist()
  for feature in top_5:
    if feature in categorical_features:
      pd_plot = wandb.plot.scatter(table = shap_wandb_table, x = feature, y = f"{feature}_shap", title = f"PD Plot: {feature}")
    else:
      pd_plot = wandb.plot.line(table = shap_wandb_table, x = feature, y = f"{feature}_shap", title = f"PD Plot: {feature}")
    run.log({ f"Shap Partial Dependence Plot {feature}": pd_plot})
  run.log({"Shap Feature Importance": shap_fi_plot})
  shap_values_df["Shap VIZ"] = shap_html_list

  run.log({"Shap Values": wandb.Table(dataframe = shap_values_df)})


Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


[0]	validation_0-auc:0.74603	validation_0-logloss:0.50882
[1]	validation_0-auc:0.75616	validation_0-logloss:0.49454
[2]	validation_0-auc:0.76581	validation_0-logloss:0.48310


`use_label_encoder` is deprecated in 1.7.0.
`callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.


[3]	validation_0-auc:0.76847	validation_0-logloss:0.47404
[4]	validation_0-auc:0.77308	validation_0-logloss:0.46673
[5]	validation_0-auc:0.77510	validation_0-logloss:0.46052
[6]	validation_0-auc:0.77469	validation_0-logloss:0.45557
[7]	validation_0-auc:0.77533	validation_0-logloss:0.45137
[8]	validation_0-auc:0.77599	validation_0-logloss:0.44772
[9]	validation_0-auc:0.77758	validation_0-logloss:0.44432
[10]	validation_0-auc:0.77809	validation_0-logloss:0.44169
[11]	validation_0-auc:0.77874	validation_0-logloss:0.43956
[12]	validation_0-auc:0.77915	validation_0-logloss:0.43754
[13]	validation_0-auc:0.78006	validation_0-logloss:0.43594
[14]	validation_0-auc:0.77978	validation_0-logloss:0.43465
[15]	validation_0-auc:0.77970	validation_0-logloss:0.43334
[16]	validation_0-auc:0.78040	validation_0-logloss:0.43214
[17]	validation_0-auc:0.78065	validation_0-logloss:0.43115
[18]	validation_0-auc:0.78065	validation_0-logloss:0.43036
[19]	validation_0-auc:0.78075	validation_0-logloss:0.42967
[20]

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
validation_0-auc,▁▃▅▅▆▇▇▇▇▇▇▇▇██▇█████████
validation_0-logloss,█▇▆▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch,24


## Sweeps

Very Naive instrumentation of sweeps - sweeps running a single machine

We'll switch from using the `sync_tensorboard` argument to using `wandb.log`.  

Also, we'll use sweeps to evaluate different batch sizes and learning rates.  

1. Add wandb: In your Python script, add a couple lines of code to log hyperparameters and output metrics from your script.
2. Write config: Define the variables and ranges to sweep over. Pick a search strategy— we support grid, random, and Bayesian search, plus techniques for faster iterations like early stopping. Check out some example configs here.
Initialize sweep: Launch the sweep server. We host this central controller and coordinate between the agents that execute the sweep.
3. Launch agent(s): Run a single-line command on each machine you'd like to use to train models in the sweep. The agents ask the central sweep server what hyperparameters to try next, and then they execute the runs.
4. Visualize results: Open our live dashboard to see all your results in one central place.

![](https://1039519455-files.gitbook.io/~/files/v0/b/gitbook-x-prod.appspot.com/o/spaces%2F-Lqya5RvLedGEWPhtkjU-1972196547%2Fuploads%2Fgit-blob-d7820a5646e118213a46afd4faa2c02eed7faf5c%2Fcentral-sweep-server-3%20(2)%20(2)%20(3)%20(3)%20(2)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(3)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(2).png?alt=media)

## Sweep Configuration

In [29]:
sweep_config = {
  'method': 'grid',
  'metric': {
      'name': 'validation_0-logloss.min',  ## matches what i write via SummaryWriter
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
      'feature_list':{
          'values': ["fls1", "fls2"]
      },
      'max_depth': {
          'values': [2, 4, 16]
      },
      "colsample_bytree": {
          "values": [0.6, 0.8, 1]
      }
  }
}


## Train Fucntion

In [30]:
def sweep_train(config_defaults = dict(learning_rate=0.01, max_depth = 3, n_estimators = 25)):

  config_standard = {
        'objective': 'binary:logistic'
        , 'gamma': 1               ## def: 0
        , 'min_child_weight': 100  ## def: 1
        , 'nthread': 24
        , 'random_state': 42
        , 'reg_alpha': 0
        , 'reg_lambda': 0          ## def: 1
        , 'eval_metric': ['auc', 'logloss']
        , 'tree_method': 'hist'  # use `gpu_hist` to train on GPU,
        , 'n_estimators': 25
    }

  config = {**config_defaults, **config_standard}

  with wandb.init(config = config) as run:

    config = wandb.config
    ## or, ifyou have a nasty nested dictionary for your config
    # config = EasyDict(wandb.config)

    # artifact = run.use_artifact('{entity}/{project_name}/train-val-test-datasets:v0', type='dataset')
    artifact = run.use_artifact('tim-w/xgboost-example/train-val-test-datasets:v0', type='dataset')
    artifact_path = artifact.download()

    categorical_features = ["EDUCATION", "MARRIAGE", "PAY_1", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"]
    numeric_features = ['LIMIT_BAL', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3',
          'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2',
          'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
    features = []
    features.extend(numeric_features)
    features.extend(categorical_features)
    for col in categorical_features:
      dataset[col] = dataset[col].astype('category')

    df_train = pd.read_csv(artifact_path + "/train.csv")
    df_val = pd.read_csv(artifact_path + "/val.csv")
    df_test = pd.read_csv(artifact_path + "/test.csv")

    X_train = df_train.drop([target_name], axis=1)
    Y_train = df_train[target_name]
    X_test = df_test.drop([target_name], axis=1)
    Y_test = df_test[target_name]
    X_val = df_val.drop([target_name], axis=1)
    Y_val = df_val[target_name]
    base_rate = Y_train.mean()

    config['base_score'] = base_rate

    early_stopping_rounds = 40

    pipeline = create_preprocessing_pipeline(numeric_features, categorical_features)
    pipeline.fit(X_train)

    X_train = pipeline.transform(X_train)
    X_val = pipeline.transform(X_val)
    X_test = pipeline.transform(X_test)

    X_train = pd.DataFrame(X_train, columns = features)
    X_val = pd.DataFrame(X_val, columns = features)
    X_test = pd.DataFrame(X_test, columns = features)

    xgbmodel = xgb.XGBClassifier(**wandb.config, use_label_encoder=False)
    xgbmodel.fit(X_train, Y_train, eval_set = [(X_val, Y_val)],
                callbacks = [WandbCallback(log_model = True)]
                )

## Initialize Sweep

In [31]:
sweep_id = wandb.sweep(sweep_config, project=project_name)

Create sweep with ID: bsuh7a0v
Sweep URL: https://wandb.ai/tim-w/xgboost-example/sweeps/bsuh7a0v


## Run the Sweep Agent

In [32]:
wandb_agent = wandb.agent(sweep_id, function=sweep_train)

wandb: Agent Starting Run: iffl5gul with config:
wandb: 	colsample_bytree: 0.6
wandb: 	feature_list: fls1
wandb: 	max_depth: 2


wandb:   3 of 3 files downloaded.  
`use_label_encoder` is deprecated in 1.7.0.
`callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.


[0]	validation_0-auc:0.70763	validation_0-logloss:0.52637
[1]	validation_0-auc:0.73998	validation_0-logloss:0.52505
[2]	validation_0-auc:0.74337	validation_0-logloss:0.52321
[3]	validation_0-auc:0.74378	validation_0-logloss:0.52142
[4]	validation_0-auc:0.75397	validation_0-logloss:0.52019
[5]	validation_0-auc:0.75619	validation_0-logloss:0.51849
[6]	validation_0-auc:0.76020	validation_0-logloss:0.51687
[7]	validation_0-auc:0.76611	validation_0-logloss:0.51591
[8]	validation_0-auc:0.76510	validation_0-logloss:0.51480
[9]	validation_0-auc:0.76558	validation_0-logloss:0.51323
[10]	validation_0-auc:0.76615	validation_0-logloss:0.51218
[11]	validation_0-auc:0.76492	validation_0-logloss:0.51114
[12]	validation_0-auc:0.76579	validation_0-logloss:0.50967
[13]	validation_0-auc:0.76483	validation_0-logloss:0.50868
[14]	validation_0-auc:0.76511	validation_0-logloss:0.50728
[15]	validation_0-auc:0.76651	validation_0-logloss:0.50592
[16]	validation_0-auc:0.76715	validation_0-logloss:0.50516
[17]	va

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
validation_0-auc,▁▅▅▅▆▆▇▇▇▇▇▇▇▇▇▇█▇███████
validation_0-logloss,██▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁▁
epoch,24


wandb: Agent Starting Run: rn2v2inn with config:
wandb: 	colsample_bytree: 0.6
wandb: 	feature_list: fls1
wandb: 	max_depth: 4


...

wandb: Agent Starting Run: v3sl15kd with config:
wandb: 	colsample_bytree: 1
wandb: 	feature_list: fls2
wandb: 	max_depth: 16


wandb:   3 of 3 files downloaded.  
`use_label_encoder` is deprecated in 1.7.0.
`callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.


[0]	validation_0-auc:0.76429	validation_0-logloss:0.52617
[1]	validation_0-auc:0.76448	validation_0-logloss:0.52409
[2]	validation_0-auc:0.76446	validation_0-logloss:0.52207
[3]	validation_0-auc:0.76469	validation_0-logloss:0.52011
[4]	validation_0-auc:0.76549	validation_0-logloss:0.51820
[5]	validation_0-auc:0.76729	validation_0-logloss:0.51634
[6]	validation_0-auc:0.76783	validation_0-logloss:0.51452
[7]	validation_0-auc:0.76892	validation_0-logloss:0.51276
[8]	validation_0-auc:0.76906	validation_0-logloss:0.51106
[9]	validation_0-auc:0.76958	validation_0-logloss:0.50940
[10]	validation_0-auc:0.76946	validation_0-logloss:0.50779
[11]	validation_0-auc:0.76990	validation_0-logloss:0.50621
[12]	validation_0-auc:0.76983	validation_0-logloss:0.50468
[13]	validation_0-auc:0.77032	validation_0-logloss:0.50317
[14]	validation_0-auc:0.77047	validation_0-logloss:0.50171
[15]	validation_0-auc:0.77105	validation_0-logloss:0.50026
[16]	validation_0-auc:0.77128	validation_0-logloss:0.49884
[17]	va

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
validation_0-auc,▁▁▁▁▂▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇█
validation_0-logloss,██▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁▁
epoch,24


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## Use API to interact with W&B

In [ ]:
import pandas as pd
import wandb
api = wandb.Api()
sweep = api.sweep(f"{entity}/{project_name}/{sweep_id}")
temp_data = []
for r in sweep.runs:
 temp_dict = dict(**dict(r.summary), **r.config)
 temp_dict["run_id"] = r.id
 temp_dict["run_name"] = r.name
 temp_data.append( temp_dict)
df = pd.DataFrame(temp_data)
df.set_index("run_id", inplace = True)
best_run_id = sweep.best_run().id
best_run = api.run(f"{entity}/{project_name}/{best_run_id}")
df.loc[best_run_id]

wandb: Sorting runs by +summary_metrics.validation_0-logloss.min


_wandb                                                         {'runtime': 2}
_runtime                                                             2.470428
_timestamp                                                  1661181702.512003
validation_0-auc                                             {'max': 0.77826}
validation_0-logloss                                        {'min': 0.439419}
Feature Importance_table    {'_type': 'table-file', 'ncols': 2, 'nrows': 1...
_step                                                                      25
epoch                                                                      24
gamma                                                                       1
learner                     {'metrics': ['auc', 'logloss'], 'objective': {...
nthread                                                                    24
version                                                             [1, 3, 3]
max_depth                                                       